In [1]:
%load_ext autoreload
%autoreload 2

# Schema Validation

In this notebook, we will implement TFX's schema validation to see if any of the corruptions in the previous notebook were detected. We'll log the results of schema validation to wandb.

You can set up wandb alerts here: https://docs.wandb.ai/guides/runs/alert

I use Modal because TFDV doesn't run on Mac M1s. You can create a free account on Modal here: https://modal.com/signup -- it comes with $10/month credits, which should be plenty more than enough to run the notebooks in this course. Once you have created an account, follow the "Getting Started" instructions on the homepage:

* Run `pip install modal-client` (also included in `requirements.txt` in this repo)
* Run `modal token new`, which will open a browser window and authenticate you with your account

Then you should be able to run this notebook!

In [4]:
from dataval.dataset import WeatherDataset
from dataval.train import CatBoostTrainer

import os
import matplotlib.pyplot as plt
import modal
import pandas as pd

import wandb
from wandb import AlertLevel

In [5]:
image = (
    modal.Image.debian_slim()
    .pip_install_from_requirements("requirements.txt")
    .pip_install(["tensorflow-data-validation", "tensorflow_metadata"])
)
stub = modal.Stub("tfdv-tutorial", image=image)

In [6]:
# Load dataset

ds = WeatherDataset(os.path.join(os.getcwd(), "canonical-partitioned-dataset"), sample_frac=0.2)

In [7]:
train_df = ds.load(ds.get_partition_keys()[0])
test_df = ds.load(ds.get_partition_keys()[1])

## Infer schema and check test data for errors

From the train dataframe, we create a schema using TFDV. Then we use this schema to find anomalies in the test data. We apply this to the original dataframes first.

In [8]:
@stub.function
def find_anomalies(train_df, test_df):
    import tensorflow_data_validation as tfdv
    from google.protobuf.json_format import MessageToDict
    
    train_stats =  tfdv.generate_statistics_from_dataframe(train_df)
    schema = tfdv.infer_schema(statistics=train_stats)
    test_stats = tfdv.generate_statistics_from_dataframe(test_df)
    # tfdv.visualize_statistics(lhs_statistics=test_stats, rhs_statistics=train_stats, lhs_name='TEST_DATASET', rhs_name='TRAIN_DATASET') TODO: figure out how to run this
    anomalies = tfdv.validate_statistics(statistics=test_stats, schema=schema)
    anomalies_df = tfdv.utils.display_util.get_anomalies_dataframe(anomalies)
    # return MessageToDict(anomalies)
    return anomalies_df

In [9]:
with stub.run():
    X_train, _ = ds.split_feature_label(train_df)
    X_test, _ = ds.split_feature_label(test_df)
    anomalies = find_anomalies.call(X_train, X_test)

Output()

✓ Initialized. View app at https://modal.com/apps/ap-FOsx9peIWicnn67oNyGVIA

Output()

Building image im-xxkuadsrsgdopllbonxycxklhlji



=> Step 0: FROM base


...

2023-05-01 00:06:36.175129: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.


2023-05-01 00:06:37.169338: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.


2023-05-01 00:06:37.171421: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2023-05-01 00:06:47.561666: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


✓ App completed.

In [10]:
anomalies

,Anomaly short description,Anomaly long description
Feature name,,


Seems like the raw data did not have any anomalies!

## Iterate through corruptions

See if tfdv detects any anomalies, for all the corruptions we had in our previous notebook.

In [11]:
X_train, _ = ds.split_feature_label(train_df)
corruption_anomalies = {}

with stub.run():
    for corruption_name, corruption_res in ds.iterate_corruptions(test_df, "cmc", corruption_rate=0.05):
        corrupted_test_df, corrupted_columns = corruption_res
        corrupted_X_test, _ = ds.split_feature_label(corrupted_test_df)
        corruption_anomalies[corruption_name] = find_anomalies.call(X_train, corrupted_X_test)

Output()

✓ Initialized. View app at https://modal.com/apps/ap-Ks3YcZu5qHoTAWyxDnDCi2

Output()

✓ Created objects.
└── 🔨 Created find_anomalies.

Output()

...

2023-05-01 00:07:48.458427: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.


2023-05-01 00:07:48.634047: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.


2023-05-01 00:07:48.634401: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2023-05-01 00:07:54.158376: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


✓ App completed.

In [10]:
# Send wandb alerts

run = wandb.init(project="ml-dataval-tutorial", tags=["TFDV-schema"])

for corruption_name, anomalies in corruption_anomalies.items():
    if len(anomalies) > 0:
        table = wandb.Table(dataframe=anomalies)
        wandb.log({corruption_name: table})
    
        wandb.alert(
            title=f"Errors detected in {corruption_name} experiment", 
            # text=f"Found the following anomalies: {anomalies.to_string()}",
            text = f"Found {len(anomalies)} anomalies",
            level=AlertLevel.WARN,
        )

wandb.finish()

wandb: Currently logged in as: sh_reya (nnprov). Use `wandb login --relogin` to force relogin


## Takeaways

It looks like only the `corrupt_null` corruption was flagged by schema validation! Maybe other validation techniques might flag them. Nevertheless, all the corruptions that schema validation found were true corruptions, so there isn't a false positive alert problem here.